In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import PowerTransformer
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV


from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from worldcuptransformers import CustomTransformer

# Dataset Preparation
Read the dataset and clean datas. Remove outliers and transform features.
Create target with multi ouptu regressor problem.

In [2]:
# define constant & read the dataset
FOLDER = 'dataset/'
FILENAME = 'training_dataset.csv'

df = pd.read_csv(f'{FOLDER}{FILENAME}')
df.shape

(4371, 20)

In [3]:
df.columns.tolist()
df['matchdate']=pd.to_datetime(df['matchday'], format='%Y-%m-%d')
df

,Unnamed: 0,matchday,home_team,away_team,season,competition,correct_score,1N2_1,1N2_N,1N2_2,...,away_team_rank_FIFA,home_team_trend,away_team_trend,home_team_odd_trend,away_team_odd_trend,target,best_odd_won,corr_score_home,corr_score_away,matchdate
0,0,2002-05-31,France,Senegal,2002,world-cup,0:1,1.33,4.28,8.63,...,38.454545,-1.0,-1.0,-1.0000,-1.0000,2.0,False,0.999099,0.984973,2002-05-31
1,1,2002-06-01,Uruguay,Denmark,2002,world-cup,1:2,2.65,2.72,2.97,...,15.727273,-1.0,-1.0,-1.0000,4.3675,2.0,False,0.785714,0.988381,2002-06-01
2,2,2002-06-01,Ireland,Cameroon,2002,world-cup,1:1,3.33,2.90,2.28,...,19.090909,8.0,-1.0,2.1125,5.7825,1.0,False,0.881049,1.000000,2002-06-01
3,3,2002-06-01,Germany,Saudi Arabia,2002,world-cup,8:0,1.29,4.73,8.73,...,35.818182,10.0,10.0,2.5325,2.1125,0.0,True,0.989398,0.858464,2002-06-01
4,4,2002-06-02,Spain,Slovenia,2002,world-cup,3:1,1.42,4.10,7.57,...,30.909091,11.0,6.0,1.7250,2.2550,0.0,True,0.973360,0.952510,2002-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,4366,2021-07-03,Czech Republic,Denmark,2020,euro,1:2,3.94,3.08,2.13,...,10.666667,9.0,9.0,1.9425,2.0625,2.0,True,0.884615,0.988381,2021-07-03
4367,4367,2021-07-03,Ukraine,England,2020,euro,0:4,7.64,3.98,1.52,...,4.000000,12.0,12.0,1.8250,2.0300,2.0,True,0.956146,0.949018,2021-07-03
4368,4368,2021-07-06,Italy,Spain,2020,euro,2:1,2.50,3.14,3.10,...,6.000000,9.0,9.0,2.0100,1.9175,0.0,True,0.873981,0.973360,2021-07-06
4369,4369,2021-07-07,England,Denmark,2020,euro,2:1,1.75,3.44,5.58,...,10.666667,12.0,9.0,2.0675,2.1250,0.0,True,0.949018,0.988381,2021-07-07


In [4]:
features = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','away_team_rank_FIFA','season','home_team','away_team']
target = ['home_score','away_score']
for idx, r in df.iterrows():
    s = r.correct_score.split(':')
    df.loc[idx,'home_score']=int(s[0])
    df.loc[idx,'away_score']=int(s[1]) 
df.drop('correct_score', axis=1, inplace=True)    


In [5]:
trainset = df.query('competition != "world-cup" or (competition == "world-cup" and season < 2014)').drop('competition', axis=1)[features+target]
testset = df.query('competition == "world-cup" and (season <= 2014)').drop('competition', axis=1)[features+target]
unseenset = df.query('competition == "world-cup" and (season == 2018)').drop('competition', axis=1)[features+target]

wc2018 =df.query('competition == "world-cup" and (season == 2018)')



In [6]:
trainset.shape, testset.shape, unseenset.shape

((4291, 10), (193, 10), (64, 10))

In [7]:
#trainset.dtypes, testset.dtypes, unseenset.dtypes

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
train_X = trainset.drop(['home_score','away_score'],axis=1).values
train_y = trainset[['home_score', 'away_score']].values


test_X = testset.drop(['home_score','away_score'],axis=1).values
test_y = testset[['home_score', 'away_score']].values


unseen_X = unseenset.drop(['home_score','away_score'],axis=1).values



In [10]:
trainset.drop(['home_score','away_score'], axis=1).columns
              

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team'],
      dtype='object')

In [11]:
regressor = KNeighborsRegressor()


import warnings
warnings.filterwarnings("ignore")
from sklearn.exceptions import FitFailedWarning 

In [12]:
trainset.columns

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team', 'home_score', 'away_score'],
      dtype='object')

In [13]:
regressor = DecisionTreeRegressor()

train_X = trainset.drop(['home_score','away_score'],axis=1)
train_y = trainset[['home_score', 'away_score']]

test_X = testset.drop(['home_score','away_score'],axis=1)
test_y = testset[['home_score', 'away_score']]

numeric_features = ['1N2_1', '1N2_N', '1N2_2', 'away_team_rank_FIFA', 'home_team_rank_FIFA',
       'season']
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ['home_team','away_team']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


clf = Pipeline(
    steps=[
        ('experimental_trans', CustomTransformer('1N2_1')) ,       
        ("preprocessor", preprocessor), 

        ("classifier", regressor)
        ]
)


clf.fit(train_X, train_y)
print("model score: %.3f" % clf.score(test_X, test_y))

import joblib

#save your model or results
joblib.dump(clf,'models/multi-regressor-score.pkl')

#load your model for further usage
model = joblib.load("models/multi-regressor-score.pkl")

model score: 0.793


In [14]:
clf

Pipeline(steps=[('experimental_trans', CustomTransformer(feature_name='1N2_1')),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['1N2_1', '1N2_N', '1N2_2',
                                                   'away_team_rank_FIFA',
                                                   'home_team_rank_FIFA',
                                                   'season']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                

In [15]:
model.predict(test_X.iloc[0:3,:])

array([[0., 1.],
       [1., 2.],
       [1., 1.]])

In [16]:
grid = clf

In [17]:
trainset.columns

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team', 'home_score', 'away_score'],
      dtype='object')

In [18]:
train_X.dtypes

1N2_1                  float64
1N2_N                  float64
1N2_2                  float64
home_team_rank_FIFA    float64
away_team_rank_FIFA    float64
season                   int64
home_team               object
away_team               object
dtype: object

In [19]:
test_X

,1N2_1,1N2_N,1N2_2,home_team_rank_FIFA,away_team_rank_FIFA,season,home_team,away_team
0,1.33,4.28,8.63,1.818182,38.454545,2002,France,Senegal
1,2.65,2.72,2.97,23.545455,15.727273,2002,Uruguay,Denmark
2,3.33,2.90,2.28,54.727273,19.090909,2002,Ireland,Cameroon
3,1.29,4.73,8.73,7.545455,35.818182,2002,Germany,Saudi Arabia
4,1.42,4.10,7.57,5.090909,30.909091,2002,Spain,Slovenia
...,...,...,...,...,...,...,...,...
2283,1.52,4.13,7.02,8.333333,24.250000,2014,Netherlands,Costa Rica
2284,2.64,3.15,2.91,6.583333,1.500000,2014,Brazil,Germany
2285,3.03,3.20,2.52,8.333333,3.250000,2014,Netherlands,Argentina
2286,2.16,3.67,3.28,6.583333,8.333333,2014,Brazil,Netherlands


In [20]:
model.predict(test_X.iloc[0:1,:])

array([[0., 1.]])

In [21]:
#pip install xgboost

In [22]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor


In [23]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

In [24]:
train_X

,1N2_1,1N2_N,1N2_2,home_team_rank_FIFA,away_team_rank_FIFA,season,home_team,away_team
0,1.33,4.28,8.63,1.818182,38.454545,2002,France,Senegal
1,2.65,2.72,2.97,23.545455,15.727273,2002,Uruguay,Denmark
2,3.33,2.90,2.28,54.727273,19.090909,2002,Ireland,Cameroon
3,1.29,4.73,8.73,7.545455,35.818182,2002,Germany,Saudi Arabia
4,1.42,4.10,7.57,5.090909,30.909091,2002,Spain,Slovenia
...,...,...,...,...,...,...,...,...
4366,3.94,3.08,2.13,40.666667,10.666667,2020,Czech Republic,Denmark
4367,7.64,3.98,1.52,24.000000,4.000000,2020,Ukraine,England
4368,2.50,3.14,3.10,8.000000,6.000000,2020,Italy,Spain
4369,1.75,3.44,5.58,4.000000,10.666667,2020,England,Denmark


In [25]:
train_y

,home_score,away_score
0,0.0,1.0
1,1.0,2.0
2,1.0,1.0
3,8.0,0.0
4,3.0,1.0
...,...,...
4366,1.0,2.0
4367,0.0,4.0
4368,2.0,1.0
4369,2.0,1.0


In [29]:
train_X.iloc[:,0:6]

,1N2_1,1N2_N,1N2_2,home_team_rank_FIFA,away_team_rank_FIFA,season
0,1.33,4.28,8.63,1.818182,38.454545,2002
1,2.65,2.72,2.97,23.545455,15.727273,2002
2,3.33,2.90,2.28,54.727273,19.090909,2002
3,1.29,4.73,8.73,7.545455,35.818182,2002
4,1.42,4.10,7.57,5.090909,30.909091,2002
...,...,...,...,...,...,...
4366,3.94,3.08,2.13,40.666667,10.666667,2020
4367,7.64,3.98,1.52,24.000000,4.000000,2020
4368,2.50,3.14,3.10,8.000000,6.000000,2020
4369,1.75,3.44,5.58,4.000000,10.666667,2020


In [31]:
estimator = XGBRegressor(objective='reg:squarederror')
model = MultiOutputRegressor(estimator=estimator).fit(train_X.iloc[:,0:6], train_y)


In [39]:
np.round(model.predict(test_X.iloc[0:4,0:6]))

array([[0., 1.],
       [1., 2.],
       [1., 1.],
       [6., 0.]], dtype=float32)

In [35]:
test_y.iloc[0:4,:]

,home_score,away_score
0,0.0,1.0
1,1.0,2.0
2,1.0,1.0
3,8.0,0.0


In [ ]:
#save your model or results
joblib.dump(model,'models/multi-regressor-score.pkl')

#load your model for further usage
model = joblib.load("models/multi-regressor-score.pkl")